In [0]:
import pandas as pd

In [2]:
df_train = pd.read_csv('sales_train.csv', sep=',')
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [0]:
df_train.shape

(2935849, 6)

In [0]:
p = df_train['item_id'].unique()
len(p)

21807

In [0]:
def intersection(a, b):

  lst = [value for value in a if value in b]
  return lst

In [0]:
teste = intersection(p, itens)
len(teste)

4737

In [0]:
df_train.shape

(2935849, 6)

In [3]:
# a coluna item_cnt_day não pode conter valores negativos pois representa a quantidade de produtos vendidos. Deve haver erros nessa coluna, então isso deve ser consertado.

# função para transformar um numero negativo em positivo

def positive (num):
    
    if num < 0:
        num = num*(-1)
        
    return num

# aplicando a função na coluna

df_train['item_cnt_day'] = df_train['item_cnt_day'].apply(positive)
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
df_items = pd.read_csv('items.csv', sep=',')
df_items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [5]:
df_shops = pd.read_csv('shops.csv', sep=',')
df_shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [6]:
group = pd.DataFrame(df_train.groupby(['item_id', 'date_block_num'])['item_cnt_day'].sum().rename('monthly_label'))
group.head()

monthly_label
item_id date_block_num               
0       20                        1.0
1       15                        2.0
        18                        1.0
        19                        1.0
        20                        1.0

In [0]:
group.shape

(233912, 1)

In [0]:
group['monthly_label'].describe()

count    233912.000000
mean         15.660967
std          96.194809
min           1.000000
25%           2.000000
50%           4.000000
75%          11.000000
max       12557.000000
Name: monthly_label, dtype: float64

In [7]:
df_prices = df_train[['item_id', 'date_block_num', 'item_price']]
df_prices.head()

,item_id,date_block_num,item_price
0,22154,0,999.00
1,2552,0,899.00
2,2552,0,899.00
3,2554,0,1709.05
4,2555,0,1099.00


In [8]:
# juntar os dataframes 'group' e 'df_prices' avaliar a correlação entre preço e vendas

prices = pd.merge(group, df_prices, on=['item_id', 'date_block_num'])
prices.head()

,item_id,date_block_num,monthly_label,item_price
0,0,20,1.0,58.0
1,1,15,2.0,4490.0
2,1,15,2.0,4490.0
3,1,18,1.0,4490.0
4,1,19,1.0,4490.0


In [9]:
# é preciso deletar as informações duplicadas de um mesmo item em um mesmo mês. O que interessa é somente a informação de cada item em cada mês
prices = prices.drop_duplicates(subset =['item_id', 'date_block_num'])
prices.shape

(233912, 4)

In [0]:
prices['item_price'].describe()

count    233912.000000
mean        677.105644
std        1522.402433
min           0.100000
25%         180.250000
50%         299.000000
75%         699.000000
max      307980.000000
Name: item_price, dtype: float64

In [0]:
shop = df_train[['item_id', 'date_block_num', 'shop_id']]

In [11]:
# juntar os dataframes 'prices' e 'shop' para agregar a coluna shop_id

data = pd.merge(prices, shop, on=['item_id', 'date_block_num'])
data.head()

,item_id,date_block_num,monthly_label,item_price,shop_id
0,0,20,1.0,58.0,54
1,1,15,2.0,4490.0,55
2,1,15,2.0,4490.0,55
3,1,18,1.0,4490.0,55
4,1,19,1.0,4490.0,55


In [12]:
# é preciso deletar as informações duplicadas de um mesmo item em um mesmo mês. O que interessa é somente a informação de cada item em cada mês
data = data.drop_duplicates(subset =['item_id', 'date_block_num'])
data.shape

(233912, 5)

In [0]:
category = df_items[['item_id', 'item_category_id']]

In [100]:
# agregando a coluna 'item_category_id' ao dataframe

data_train = pd.merge(data, category, on=['item_id'])
data_train.shape

(233912, 6)

In [101]:
data_train = data_train[['item_id', 'item_category_id', 'date_block_num', 'item_price', 'shop_id', 'monthly_label']]
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label
0,0,40,20,58.0,54,1.0
1,1,76,15,4490.0,55,2.0
2,1,76,18,4490.0,55,1.0
3,1,76,19,4490.0,55,1.0
4,1,76,20,4490.0,55,1.0


In [102]:
# sorting the dataset by date_block_num

data_train = data_train.sort_values(by=['date_block_num', 'item_id'], ascending=True)
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label
26,19,40,0,28.0,25,1.0
34,27,19,0,2499.0,19,7.0
50,28,30,0,549.0,15,8.0
63,29,23,0,2499.0,28,6.0
136,32,40,0,349.0,24,299.0


In [103]:
# basic feature engineering
# function to calculate last month sales and the difference between last month and current month
data_train['Last_Month_Sales'] = data_train.groupby(['item_id'])['monthly_label'].shift()
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales
26,19,40,0,28.0,25,1.0,NaN
34,27,19,0,2499.0,19,7.0,NaN
50,28,30,0,549.0,15,8.0,NaN
63,29,23,0,2499.0,28,6.0,NaN
136,32,40,0,349.0,24,299.0,NaN


In [104]:
data_train = data_train.fillna(0)
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales
26,19,40,0,28.0,25,1.0,0.0
34,27,19,0,2499.0,19,7.0,0.0
50,28,30,0,549.0,15,8.0,0.0
63,29,23,0,2499.0,28,6.0,0.0
136,32,40,0,349.0,24,299.0,0.0


# Pushing hard the feature engineering process

In [0]:
# creating columns with max, min and mean monthly sales by item_id

data_train = pd.merge(data_train, pd.DataFrame(data_train.groupby(['item_id'])['monthly_label'].max().rename('month_max_sales')).reset_index(), on='item_id')
data_train = pd.merge(data_train, pd.DataFrame(data_train.groupby(['item_id'])['monthly_label'].min().rename('month_min_sales')).reset_index(), on='item_id')
data_train = pd.merge(data_train, pd.DataFrame(data_train.groupby(['item_id'])['monthly_label'].mean().rename('month_average_sales')).reset_index(), on='item_id')

In [109]:
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales
0,19,40,0,28.0,25,1.0,0.0,1.0,1.0,1.000
1,27,19,0,2499.0,19,7.0,0.0,7.0,1.0,2.625
2,27,19,1,1998.0,56,3.0,7.0,7.0,1.0,2.625
3,27,19,2,1998.0,32,4.0,3.0,7.0,1.0,2.625
4,27,19,3,1998.0,14,1.0,4.0,7.0,1.0,2.625


In [0]:
# creating columns with max, min and mean monthly sales by shop_id

data_train = pd.merge(data_train, pd.DataFrame(data_train.groupby(['shop_id'])['monthly_label'].max().rename('shop_max_sales')).reset_index(), on='shop_id')
data_train = pd.merge(data_train, pd.DataFrame(data_train.groupby(['shop_id'])['monthly_label'].min().rename('shop_min_sales')).reset_index(), on='shop_id')
data_train = pd.merge(data_train, pd.DataFrame(data_train.groupby(['shop_id'])['monthly_label'].mean().rename('shop_average_sales')).reset_index(), on='shop_id')

In [111]:
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales
0,19,40,0,28.0,25,1.0,0.0,1.0,1.0,1.000000,12557.0,1.0,11.670768
1,33,37,3,499.0,25,16.0,32.0,61.0,11.0,24.823529,12557.0,1.0,11.670768
2,34,40,3,149.0,25,12.0,8.0,15.0,1.0,6.777778,12557.0,1.0,11.670768
3,35,40,0,399.0,25,78.0,0.0,119.0,1.0,44.400000,12557.0,1.0,11.670768
4,40,57,1,249.0,25,5.0,4.0,11.0,1.0,3.357143,12557.0,1.0,11.670768


In [0]:
# creating columns with max, min and mean monthly sales by item_category_id

data_train = pd.merge(data_train, pd.DataFrame(data_train.groupby(['item_category_id'])['monthly_label'].max().rename('category_max_sales')).reset_index(), on='item_category_id')
data_train = pd.merge(data_train, pd.DataFrame(data_train.groupby(['item_category_id'])['monthly_label'].min().rename('category_min_sales')).reset_index(), on='item_category_id')
data_train = pd.merge(data_train, pd.DataFrame(data_train.groupby(['item_category_id'])['monthly_label'].mean().rename('category_average_sales')).reset_index(), on='item_category_id')

In [114]:
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales,category_max_sales,category_min_sales,category_average_sales
0,19,40,0,28.0,25,1.0,0.0,1.0,1.0,1.000000,12557.0,1.0,11.670768,1401.0,1.0,11.855094
1,34,40,3,149.0,25,12.0,8.0,15.0,1.0,6.777778,12557.0,1.0,11.670768,1401.0,1.0,11.855094
2,35,40,0,399.0,25,78.0,0.0,119.0,1.0,44.400000,12557.0,1.0,11.670768,1401.0,1.0,11.855094
3,72,40,0,119.0,25,4.0,0.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094
4,72,40,3,119.0,25,3.0,7.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094


In [0]:
def item_shop_reason (item, shop):

  reason = item/shop

  return reason

In [118]:
# creating a rate item mean sales / shop mean sales

data_train['item_shop_rate'] = data_train.apply(lambda x: item_shop_reason(x['month_average_sales'], x['shop_average_sales']), axis=1)
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales,category_max_sales,category_min_sales,category_average_sales,item_shop_rate
0,19,40,0,28.0,25,1.0,0.0,1.0,1.0,1.000000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.085684
1,34,40,3,149.0,25,12.0,8.0,15.0,1.0,6.777778,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.580748
2,35,40,0,399.0,25,78.0,0.0,119.0,1.0,44.400000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,3.804377
3,72,40,0,119.0,25,4.0,0.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334
4,72,40,3,119.0,25,3.0,7.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334


In [0]:
def item_category_reason (item, category):

  reason = item/category

  return reason

In [121]:
# creating a rate item mean sales / category mean sales

data_train['item_category_rate'] = data_train.apply(lambda x: item_category_reason(x['month_average_sales'], x['category_average_sales']), axis=1)
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales,category_max_sales,category_min_sales,category_average_sales,item_shop_rate,item_category_rate
0,19,40,0,28.0,25,1.0,0.0,1.0,1.0,1.000000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.085684,0.084352
1,34,40,3,149.0,25,12.0,8.0,15.0,1.0,6.777778,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.580748,0.571719
2,35,40,0,399.0,25,78.0,0.0,119.0,1.0,44.400000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,3.804377,3.745226
3,72,40,0,119.0,25,4.0,0.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334,0.287789
4,72,40,3,119.0,25,3.0,7.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334,0.287789


In [0]:
def year(block_num):

    if 0 <= block_num <= 11:

        return '2013'

    if 12 <= block_num <= 23:

        return '2014'

    if 24 <= block_num <=33:

        return '2015'

In [123]:
data_train['year'] = data_train['date_block_num'].apply(year)
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales,category_max_sales,category_min_sales,category_average_sales,item_shop_rate,item_category_rate,year
0,19,40,0,28.0,25,1.0,0.0,1.0,1.0,1.000000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.085684,0.084352,2013
1,34,40,3,149.0,25,12.0,8.0,15.0,1.0,6.777778,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.580748,0.571719,2013
2,35,40,0,399.0,25,78.0,0.0,119.0,1.0,44.400000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,3.804377,3.745226,2013
3,72,40,0,119.0,25,4.0,0.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334,0.287789,2013
4,72,40,3,119.0,25,3.0,7.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334,0.287789,2013


In [0]:
def month(block_num):

    if block_num == 0 or block_num == 12 or block_num == 24:

        return 'JAN'

    if block_num == 1 or block_num == 13 or block_num == 25:

        return 'FEV'

    if block_num == 2 or block_num == 14 or block_num == 26:

        return 'MAR'

    if block_num == 3 or block_num == 15 or block_num == 27:

        return 'ABR'

    if block_num == 4 or block_num == 16 or block_num == 28:

        return 'MAI'

    if block_num == 5 or block_num == 17 or block_num == 29:

        return 'JUN'

    if block_num == 6 or block_num == 18 or block_num == 30:

        return 'JUL'

    if block_num == 7 or block_num == 19 or block_num == 31:

        return 'AGO'

    if block_num == 8 or block_num == 20 or block_num == 32:

        return 'SET'

    if block_num == 9 or block_num == 21 or block_num == 33:

        return 'OUT'

    if block_num == 10 or block_num == 22: 

        return 'NOV'

    if block_num == 11 or block_num == 23:

        return 'DEZ'

In [125]:
data_train['month'] = data_train['date_block_num'].apply(month)
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales,category_max_sales,category_min_sales,category_average_sales,item_shop_rate,item_category_rate,year,month
0,19,40,0,28.0,25,1.0,0.0,1.0,1.0,1.000000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.085684,0.084352,2013,JAN
1,34,40,3,149.0,25,12.0,8.0,15.0,1.0,6.777778,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.580748,0.571719,2013,ABR
2,35,40,0,399.0,25,78.0,0.0,119.0,1.0,44.400000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,3.804377,3.745226,2013,JAN
3,72,40,0,119.0,25,4.0,0.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334,0.287789,2013,JAN
4,72,40,3,119.0,25,3.0,7.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334,0.287789,2013,ABR


In [0]:
# function to create an indicator if a month is end of year

def end_year(month):

  if month == 'DEZ':

    indicator = 1

  else:

    indicator = 0

  return indicator 

In [133]:
data_train['end_of_year'] = data_train['month'].apply(end_year)
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales,category_max_sales,category_min_sales,category_average_sales,item_shop_rate,item_category_rate,year,month,end_of_year,begin_year
0,19,40,0,28.0,25,1.0,0.0,1.0,1.0,1.000000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.085684,0.084352,2013,JAN,0,0
1,34,40,3,149.0,25,12.0,8.0,15.0,1.0,6.777778,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.580748,0.571719,2013,ABR,0,0
2,35,40,0,399.0,25,78.0,0.0,119.0,1.0,44.400000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,3.804377,3.745226,2013,JAN,0,0
3,72,40,0,119.0,25,4.0,0.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334,0.287789,2013,JAN,0,0
4,72,40,3,119.0,25,3.0,7.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334,0.287789,2013,ABR,0,0


In [0]:
# function to create an indicator if a month is begin of year

def begin_year(month):

  if month == 'JAN':

    indicator = 1

  elif month == 'FEV':

    indicator = 1

  else:

    indicator = 0

  return indicator 

In [136]:
data_train['begin_year'] = data_train['month'].apply(begin_year)
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales,category_max_sales,category_min_sales,category_average_sales,item_shop_rate,item_category_rate,year,month,end_of_year,begin_year
0,19,40,0,28.0,25,1.0,0.0,1.0,1.0,1.000000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.085684,0.084352,2013,JAN,0,1
1,34,40,3,149.0,25,12.0,8.0,15.0,1.0,6.777778,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.580748,0.571719,2013,ABR,0,0
2,35,40,0,399.0,25,78.0,0.0,119.0,1.0,44.400000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,3.804377,3.745226,2013,JAN,0,1
3,72,40,0,119.0,25,4.0,0.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334,0.287789,2013,JAN,0,1
4,72,40,3,119.0,25,3.0,7.0,10.0,1.0,3.411765,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.292334,0.287789,2013,ABR,0,0


In [156]:
# removing some outliers

data_train = data_train[data_train['monthly_label'] < 150]
data_train.shape

(230798, 22)

In [158]:
data_train.columns

Index(['item_id', 'item_category_id', 'date_block_num', 'item_price',
       'shop_id', 'monthly_label', 'Last_Month_Sales', 'month_max_sales',
       'month_min_sales', 'month_average_sales', 'shop_max_sales',
       'shop_min_sales', 'shop_average_sales', 'category_max_sales',
       'category_min_sales', 'category_average_sales', 'item_shop_rate',
       'item_category_rate', 'year', 'month', 'end_of_year', 'begin_year'],
      dtype='object')

In [161]:
data_train = data_train.sort_values(by=['date_block_num', 'item_id'], ascending=True)
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales,category_max_sales,category_min_sales,category_average_sales,item_shop_rate,item_category_rate,year,month,end_of_year,begin_year
0,19,40,0,28.0,25,1.0,0.0,1.0,1.0,1.000000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.085684,0.084352,2013,JAN,0,1
163368,27,19,0,2499.0,19,7.0,0.0,7.0,1.0,2.625000,3768.0,1.0,16.980093,6972.0,1.0,35.578400,0.154593,0.073781,2013,JAN,0,1
150269,28,30,0,549.0,15,8.0,0.0,13.0,1.0,6.615385,843.0,1.0,23.313817,7300.0,1.0,56.218160,0.283754,0.117673,2013,JAN,0,1
159643,29,23,0,2499.0,28,6.0,0.0,6.0,1.0,2.000000,6547.0,1.0,21.615015,3225.0,1.0,31.299817,0.092528,0.063898,2013,JAN,0,1
59716,33,37,0,499.0,24,61.0,0.0,61.0,11.0,24.823529,1226.0,1.0,17.800263,660.0,1.0,8.383264,1.394560,2.961082,2013,JAN,0,1


In [162]:
data_scaling = pd.DataFrame(data_train[['date_block_num', 'Last_Month_Sales', 'month_max_sales',
       'month_min_sales', 'month_average_sales', 'shop_max_sales',
       'shop_min_sales', 'shop_average_sales', 'category_max_sales',
       'category_min_sales', 'category_average_sales', 'item_shop_rate',
       'item_category_rate']])
data_scaling.head()

,date_block_num,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales,category_max_sales,category_min_sales,category_average_sales,item_shop_rate,item_category_rate
0,0,0.0,1.0,1.0,1.000000,12557.0,1.0,11.670768,1401.0,1.0,11.855094,0.085684,0.084352
163368,0,0.0,7.0,1.0,2.625000,3768.0,1.0,16.980093,6972.0,1.0,35.578400,0.154593,0.073781
150269,0,0.0,13.0,1.0,6.615385,843.0,1.0,23.313817,7300.0,1.0,56.218160,0.283754,0.117673
159643,0,0.0,6.0,1.0,2.000000,6547.0,1.0,21.615015,3225.0,1.0,31.299817,0.092528,0.063898
59716,0,0.0,61.0,11.0,24.823529,1226.0,1.0,17.800263,660.0,1.0,8.383264,1.394560,2.961082


In [0]:
# fazendo um scaling na coluna de preços dos itens

from sklearn.preprocessing import MinMaxScaler

scale = MinMaxScaler()
matrix = scale.fit_transform(data_scaling)

In [0]:
data_scaling[['date_block_num', 'Last_Month_Sales', 'month_max_sales',
       'month_min_sales', 'month_average_sales', 'shop_max_sales',
       'shop_min_sales', 'shop_average_sales', 'category_max_sales',
       'category_min_sales', 'category_average_sales', 'item_shop_rate',
       'item_category_rate']] = matrix

In [165]:
data_train[['date_block_num', 'Last_Month_Sales', 'month_max_sales',
       'month_min_sales', 'month_average_sales', 'shop_max_sales',
       'shop_min_sales', 'shop_average_sales', 'category_max_sales',
       'category_min_sales', 'category_average_sales', 'item_shop_rate',
       'item_category_rate']] = data_scaling[['date_block_num', 'Last_Month_Sales', 'month_max_sales',
       'month_min_sales', 'month_average_sales', 'shop_max_sales',
       'shop_min_sales', 'shop_average_sales', 'category_max_sales',
       'category_min_sales', 'category_average_sales', 'item_shop_rate',
       'item_category_rate']]
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,month_max_sales,month_min_sales,month_average_sales,shop_max_sales,shop_min_sales,shop_average_sales,category_max_sales,category_min_sales,category_average_sales,item_shop_rate,item_category_rate,year,month,end_of_year,begin_year
0,19,40,0.0,28.0,25,1.0,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.144800,0.111500,0.0,0.002483,0.000503,0.000566,2013,JAN,0,1
163368,27,19,0.0,2499.0,19,7.0,0.0,0.000861,0.000000,0.001775,0.299904,0.0,0.220389,0.555193,0.0,0.007909,0.001054,0.000495,2013,JAN,0,1
150269,28,30,0.0,549.0,15,8.0,0.0,0.001721,0.000000,0.006134,0.066911,0.0,0.310561,0.581316,0.0,0.012630,0.002085,0.000790,2013,JAN,0,1
159643,29,23,0.0,2499.0,28,6.0,0.0,0.000717,0.000000,0.001092,0.521268,0.0,0.286376,0.256770,0.0,0.006931,0.000558,0.000428,2013,JAN,0,1
59716,33,37,0.0,499.0,24,61.0,0.0,0.008607,0.067568,0.026025,0.097419,0.0,0.232065,0.052485,0.0,0.001689,0.010955,0.019921,2013,JAN,0,1


# Regressão Linear com scikit-learn

In [0]:
data_train.head()

,item_id,item_category_id,date_block_num,item_price,shop_id,monthly_label,Last_Month_Sales,Last_Month_Diff,year,month
2,1,76,18,0.121382,55,1.0,0.00008,0.602033,2014,JUL
3,1,76,19,0.121382,55,1.0,0.00000,0.602118,2014,AGO
4,1,76,20,0.121382,55,1.0,0.00000,0.602118,2014,SET
5,1,76,21,0.121382,55,1.0,0.00000,0.602118,2014,OUT
7,2,40,22,0.001565,54,1.0,0.00000,0.602118,2014,NOV


In [177]:
data_train.columns

Index(['item_id', 'item_category_id', 'date_block_num', 'item_price',
       'shop_id', 'monthly_label', 'Last_Month_Sales', 'month_max_sales',
       'month_min_sales', 'month_average_sales', 'shop_max_sales',
       'shop_min_sales', 'shop_average_sales', 'category_max_sales',
       'category_min_sales', 'category_average_sales', 'item_shop_rate',
       'item_category_rate', 'year', 'month', 'end_of_year', 'begin_year'],
      dtype='object')

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [0]:
y = data_train['monthly_label']
X = data_train[['item_category_id', 'date_block_num', 'item_price',
       'shop_id', 'Last_Month_Sales', 'month_max_sales',
       'month_min_sales', 'month_average_sales', 'shop_max_sales',
       'shop_min_sales', 'shop_average_sales', 'category_max_sales',
       'category_min_sales', 'category_average_sales', 'item_shop_rate',
       'item_category_rate', 'end_of_year', 'begin_year']]

In [0]:
from sklearn.compose import make_column_transformer

In [0]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['item_category_id', 'shop_id']), 
    remainder='passthrough')

In [0]:
X = column_trans.fit_transform(X)

In [0]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [0]:
from sklearn.linear_model import LinearRegression

In [215]:
lr_model = LinearRegression()

lr_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
y_pred = lr_model.predict(X_test)

In [217]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

168.41211902934654

In [218]:
import numpy as np

np.sqrt(mean_squared_error(y_test, y_pred))

12.977369495754775

# Random Forest Model sklearn

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
rf_model = RandomForestRegressor()

In [223]:
rf_model.fit(X_train, y_train)

KeyboardInterrupt: ignored

# XGBoost Regressor model

In [0]:
from xgboost import XGBRegressor
import xgboost as xgb

In [0]:
simple_xgb_model = XGBRegressor()

In [226]:
simple_xgb_model.fit(X_train, y_train)

[05:06:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
y_pred = simple_xgb_model.predict(X_test)

In [228]:
mean_squared_error(y_test, y_pred)

88.06528938744937

In [229]:
import numpy as np

np.sqrt(mean_squared_error(y_test, y_pred))

9.384310810467083